`title.search` search on `display name` field.

When you use search, each returned entity in the results lists gets an extra property called relevance_score, and the list is by default sorted in descending order of relevance_score.

If you search for a multiple-word phrase, the algorithm will treat each word separately, and rank results higher when the words appear close together. 

In [258]:
import json
from time import sleep
import requests
from re import sub
from thefuzz import process
from bs4 import BeautifulSoup
#from unidecode import unidecode: OpenAlex uses unicode characters
#https://api.openalex.org/works?filter=title.search:Baryonic+violation+of+R+parity+from+anomalous+U(1)H
def flatten(xss):
    return [x for xs in xss for x in xs]

In [236]:
file='/home/restrepo/Downloads/minciencias_noauthors.json'
oa_url='https://api.openalex.org/works?filter=title.search:'

In [237]:
f=open(file,'r')
j=json.load(f)
f.close()

In [238]:
len(j)

100

In [264]:
j[1]

{'titles': [{'title': 'La experiencia Internacional de crecimiento siglos XX-XXI lecciones para colombia',
   'lang': 'es',
   'source': 'minciencias'}],
 'updated': [{'source': 'minciencias', 'time': 1711049323}],
 'subtitle': '',
 'abstract': '',
 'keywords': [],
 'types': [{'provenance': 'minciencias',
   'source': 'minciencias',
   'type': 'Artículos'}],
 'external_ids': [{'provenance': 'minciencias',
   'source': 'minciencias',
   'id': 'ART-0000000007-14'},
  {'provenance': 'minciencias', 'source': 'COD_RH', 'id': '0000000007'},
  {'provenance': 'minciencias', 'source': 'COD_PRODUCTO', 'id': '14'}],
 'external_urls': [],
 'date_published': None,
 'year_published': None,
 'bibliographic_info': {},
 'references_count': None,
 'references': [],
 'citations_count': [],
 'citations': [],
 'author_count': 1,
 'authors': [{'full_name': '',
   'affiliations': [{'id': '65f93c212685775e89fdd8ce',
     'name': 'Grupo de Desarrollo y Equidad',
     'types': [{'source': 'minciencias', 'type':

In [265]:
i=7
results = []
found = []
titles = []
for i in range(len(j)):
    print(str(i).zfill(3),end='\r')
    sleep(0.1)
    original_title = j[i].get('titles')[0].get('title')
    #original_title = "Dirac dark matter, neutrino masses, and dark baryogenesis"
    #original_title = "Baryonic violation of R parity from anomalous U(1)H"
    #Remove special OpenAlex API characters
    title = sub('[_\|,]','',original_title).replace(' ','+')
    oa=requests.get(oa_url+title)

    if oa.status_code == 200 and oa.json().get('results'):
        results = oa.json().get('results') # higher relevance_score
        titles = [result.get('display_name') for result in results]
        #Check mathml in titles and UPDATE display_name value!
        for ii in range(len(titles)):
            if [tag.name for tag in BeautifulSoup(titles[ii]).find_all() if tag.name.find('mml:')>-1]:
                titles[ii] = BeautifulSoup(sub(r"([a-zA-Z])<", r"\1 <",titles[ii])).text.strip()
                results[ii]['display_name'] = titles[ii]
        display_name,score = process.extractOne(original_title,titles)
        result = [d for d in results if d.get('display_name')==display_name and score > 94]
        #If not author avalaible check at least for country
        if result:
            result = result[0]
            countries = [d.get('countries') for d  in result.get('authorships')]
            if countries:
                countries = flatten(countries)
            if 'CO' in list(set(countries)):
                result['minciencias'] = j[i]
                result['original_title'] = original_title
                result['score_thefuzz'] = score
                found.append(result)

/tmp/ipykernel_29282/2812613356.py:20: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  if [tag.name for tag in BeautifulSoup(titles[ii]).find_all() if tag.name.find('mml:')>-1]:


In [266]:
len(found)

41

In [267]:
import pandas as pd

In [269]:
pd.DataFrame(found)[['display_name','original_title','score_thefuzz']]

,display_name,original_title,score_thefuzz
0,La enseñanza en economía: una reflexión desde ...,La enseñanza en economía: una reflexión desde ...,100
1,La enseñanza de la economía: reflexiones acerc...,La enseñanza de la Economía: reflexiones acerc...,100
2,La enseñanza de la economía: reflexiones acerc...,La enseñanza de la economía: Reflexiones acerc...,95
3,Susceptibilidad de Aedes aegypti (Diptera: Cul...,Susceptibilidad de Aedes aegypti (Diptera: Cul...,100
4,Energía y desarrollo económico,Energía y Desarrollo Económico,100
5,La regulación de telefonía local en Colombia: ...,La Regulación de Telefonía Local en Colombia: ...,100
6,La economía experimental como aprendizaje sign...,La economía experimental como aprendizaje sign...,100
7,Aportes Del “Nonágono semiótico” Para la compr...,"Aportes del ""nonágono semiótico"" para la compr...",99
8,"La imagen visual de la identidad, entre resist...","La imagen visual de la identidad, entre resist...",100
9,Bioformulado de Beauveria bassiana (ATCC MYA-4...,Bioformulado de Beauveria bassiana (ATCC MYA-4...,100


In [222]:
process.extractOne(original_title,titles)

('Baryonic violation of R parity from anomalous U (1)H', 99)

In [225]:
score

99

In [158]:
original_title

'Incidencia del crecimiento económico en las emisiones de CO2Ñ Un análisis comparativo (1980-2008)'

In [159]:
original_title.replace(' ','+')

'Incidencia+del+crecimiento+económico+en+las+emisiones+de+CO2Ñ+Un+análisis+comparativo+(1980-2008)'

In [115]:
oa.status_code

200

In [116]:
    print(1)

1
